<a id="0"></a> <br>
Indice 
- [Librerias](#1)
- [Primera prueba google](#2)
- [Prueba Psn](#3)
- [Columnas de DF para webscrappear](#4)
- [Código webscrappeo sin necesidad de navegador abierto](#5)


<a id="1"></a> <br>
Librerias

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
# import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
from datetime import datetime
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import random
from bs4 import BeautifulSoup as bs
import requests
import re


In [2]:
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
# options.add_argument('--start-maximized') SOLO EN PC SOBREMESA si fuera necesario.

[Volver al índice](#0)

<a id="2"></a> <br>
Primera prueba google

In [11]:
driver = webdriver.Chrome(service=service, options=options)

driver.get("http://www.google.es")

time.sleep(3)

rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/span/div/div/div/div[3]/div[1]/button[1]/div')
rechazar_cookies.click()

time.sleep(3)

driver.quit()

[Volver al índice](#0)

<a id="3"></a> <br>
Webscrappeo Psn

In [3]:
df_juegos = pd.DataFrame(columns=["Titulo","Precio_original","Precio_actual","Precio con mayor rebaja"])

In [4]:
df_juegos

,Titulo,Precio_original,Precio_actual,Precio con mayor rebaja


In [ ]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc...
# Reset df
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()

df_juegos = pd.DataFrame(columns=["Titulo","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio_original_con_PSN","Precio_actual_sin_PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)
# element = WebDriverWait(driver, 10).until(EC.element_to_be_visible((By.ID, "card-51"))) intento de que sea mas funcional

driver.get("https://store.playstation.com/")


lista_tiempo = [1.0,1.5,2.0] #2.5,2.6,2.7,2.8,2.9,3,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

# Obtenemos número de juegos para scrapear
time.sleep(random.choice(lista_tiempo))
page_source = driver.page_source
soup = bs(page_source, 'lxml')
# Mostrar un numero de juegos limitado
numero_juegos = 10
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego

cont = 0
while cont <= 24:
    cont += 1
    time.sleep(random.choice(lista_tiempo))
    select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
    select_game.click()

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.
    page_source = driver.page_source
    soup = bs(page_source, 'lxml')

    # Variable por juego
    
    # Titulo
    try:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
    except:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
    
    # Precio original
    try:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
    except:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
    
    # Precio actual 
    try:
        precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
    except:
        precio_actual_sn_psn = precio_original_sn_psn
    
    # Plataforma
    try:
        plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
    except:
        plataforma = "No hay información"
    
    # Genero
    try:
        genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
    except:
        genero = "No hay información"
    
    # Compañia
    try:
        compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
    except:
        compania = "No hay información"
    
    # Lanzamiento
    try:  
        lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
    except:
        lanzamiento = "No hay información"
    
    # Idiomas
    try:
        idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
    except:
        idiomas = "No hay información"
    # Nº Calificaciones
    try:
        num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
    except:
        num_calificaciones = "No hay información"
    # Calificación PSN
    
    try:
        calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
    except:
        calificacion = "No hay información"
    try:
        calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
    except:
        calificacion_1 = "No hay información"
    try:
        calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
    except:
        calificacion_2 = "No hay información"
    try:
        calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
    except:
        calificacion_3 = "No hay información"
    try:
        calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
    except:
        calificacion_4 = "No hay información"
    try:
        calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
    except:
        calificacion_5 = "No hay información"
    
    # precio_con_mayor_rebaja = "No hay información"
    
    df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                    "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                    "Calificación 5 estrellas":calificacion_5,
                                    "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                    "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                    "Precio original sin PSN":precio_original_sn_psn,"Precio_actual sin PSN":precio_actual_sn_psn}
                                    # "Precio con mayor rebaja":precio_con_mayor_rebaja 
    # time.sleep(random.choice(lista_tiempo))

    driver.back();

    time.sleep(random.choice(lista_tiempo))
    #Comprobar error aquí
    if cont == 24:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()      
        cont = 0
    elif numero_juegos == len(df_juegos):
        break
    else:
        continue           
driver.quit()


1 minuto aprox cada juego.

Prueba identificacion numero de juegos.

In [11]:
df_juegos.head()

,Titulo,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio_original_con_PSN,Precio_actual_sin_PSN
0,01 Deadliest Zone Catch — Boat Crab & Fishing ...,PS4,"Acción, Simulación, Únicos",Midnight Works SRL,29/8/2023,Inglés,1.9,100 calificaciones,10 %,5 %,15 %,5 %,65 %,"14,99 €","8,99 €"
1,0 Degrees,PS5,"Acción, Puzzle",eastasiasoft,31/12/2021,Inglés,3.92,78 calificaciones,53 %,9 %,24 %,6 %,8 %,"4,99 €","14,99 €"
2,0 Rally Desert Race: Offroad Dirt Simulator,PS4,"Acción, Conducción/Carreras, Simulación",Midnight Works SRL,1/9/2023,Inglés,2.68,88 calificaciones,27 %,8 %,13 %,10 %,42 %,"14,99 €","8,99 €"
3,1001 Spikes,PS4,"Acción, Acción, Arcade",Nicalis Inc.,7/10/2015,Inglés,4.43,892 calificaciones,67 %,20 %,6 %,3 %,4 %,"14,99 €","14,99 €"
4,100ft Robot Golf,PS4,"Grupo, Deporte",NO GOBLIN LLC,18/9/2017,Inglés,3.42,285 calificaciones,38 %,17 %,13 %,12 %,20 %,"9,99 €","14,99 €"


[Volver al índice](#0)

<a id="4"></a> <br>
Columnas de DF para webscrappear

In [42]:
soup

<html><head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
</body></html>

In [ ]:
# Titulo
try:
    print(soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text())
except:
    print(soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text())

In [ ]:
# Precio Original sin PSN
soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()


In [31]:
#Precio Actual sin PSN
try:
    print(soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text())
except: 
    print(soup.find("span",class_="psw-t-title-m").get_text())

4,99 €


In [ ]:
# Precio original con PSN
try:
    print(soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text())
except: 
    soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text()

In [ ]:
#Precio Actual con PSN
try:
    print(soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text())
except: 
    try:
        print(soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text())
    except: 
        print(soup.find("span",class_="psw-t-title-m").get_text())

In [29]:
# Plataforma
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()


'PS4'

In [21]:
# Genero
soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()

'Aventura, Puzzle'

In [23]:
# Compañia
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()

'eastasiasoft'

In [24]:
# Lanzamiento
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()

'10/11/2021'

In [26]:
# Idiomas
soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()


'Alemán, Chino (Tradicional), Español, Francés (Francia), Inglés, Portugués (Brasil)'

In [33]:
# Calificación PSN
soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()


'2.57'

In [34]:
# Nº Calificaciones
soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()

'37 calificaciones'

In [35]:
# % 5 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()

'27 %'

In [36]:
# % 4 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()

'5 %'

In [37]:
# % 3 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()

'11 %'

In [38]:
# % 2 estrellas 
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()

'11 %'

In [39]:
# % 1 estrella
soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()

'46 %'

In [56]:
# Precio al día
fecha_webs = datetime.now()
fecha_webs = datetime.isoformat(fecha_webs)

[Volver al índice](#0)

<a id="5"></a> <br>
Código webscrappeo sin necesidad de navegador abierto

In [ ]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc...
# Reset df
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")


df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)


driver.get("https://store.playstation.com/")


lista_tiempo = [1.9,2.0,2.1,2.2]

#rechazamos cookies
time.sleep(random.choice(lista_tiempo))
rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
rechazar_cookies.click()

#vamos a la pestaña de explora
time.sleep(random.choice(lista_tiempo))
explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
explora.click()


#plat
time.sleep(random.choice(lista_tiempo))
sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
sort_plat.click()


#ordenamos
time.sleep(random.choice(lista_tiempo))
sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
sort_gam.click()

#AZ
time.sleep(random.choice(lista_tiempo))
sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
sort_az.click()

# Obtenemos número de juegos para scrapear
time.sleep(random.choice(lista_tiempo))
page_source = driver.page_source
soup = bs(page_source, 'lxml')
# Mostrar un numero de juegos limitado
numero_juegos = 1000
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego

cont = 0
while cont <= 24:
    cont += 1
    time.sleep(random.choice(lista_tiempo))
    select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
    select_game.click()

    time.sleep(random.choice(lista_tiempo))

    # Info que nos descargamos de la pagina del juego.
    page_source = driver.page_source
    soup = bs(page_source, 'html')

    # Variable por juego
    
    # Titulo
    try:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
    except:
        titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
    
    # Día y hora de webscrappeo
    
        fecha_webs = datetime.now()
        fecha_webs = datetime.isoformat(fecha_webs)
        
    # Precio original sin PSN
    try:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
    except:
        precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
        
    # Precio original con PSN
    try:
        precio_original_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'}).get_text()
    except: 
        try:
            precio_original_cn_psn = soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text()
        except:
            precio_original_cn_psn = precio_original_sn_psn
    # Precio actual sin PSN
    try:
        precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
    except:
        precio_actual_sn_psn = precio_original_sn_psn
    
    #Precio Actual con PSN
    try:
        precio_actual_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text()
    except: 
        try:
            precio_actual_cn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
        except: 
            precio_actual_cn_psn = precio_actual_sn_psn
    
    # Plataforma
    try:
        plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
    except:
        plataforma = "No hay información"
    
    # Genero
    try:
        genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
    except:
        genero = "No hay información"
    
    # Compañia
    try:
        compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
    except:
        compania = "No hay información"
    
    # Lanzamiento
    try:  
        lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
    except:
        lanzamiento = "No hay información"
    
    # Idiomas
    try:
        idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
    except:
        idiomas = "No hay información"
    # Nº Calificaciones
    try:
        num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
    except:
        num_calificaciones = "No hay información"
    # Calificación PSN
    
    try:
        calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
    except:
        calificacion = "No hay información"
    try:
        calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
    except:
        calificacion_1 = "No hay información"
    try:
        calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
    except:
        calificacion_2 = "No hay información"
    try:
        calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
    except:
        calificacion_3 = "No hay información"
    try:
        calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
    except:
        calificacion_4 = "No hay información"
    try:
        calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
    except:
        calificacion_5 = "No hay información"
    
    # precio_con_mayor_rebaja = "No hay información"
    
    df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                    "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                    "Calificación 5 estrellas":calificacion_5,
                                    "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                    "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                    "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn}
                                    # "Precio con mayor rebaja":precio_con_mayor_rebaja 
    # time.sleep(random.choice(lista_tiempo))

    driver.back();

    time.sleep(random.choice(lista_tiempo))
    #Comprobar error aquí
    if cont == 24:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()
        time.sleep(random.choice(lista_tiempo))      
        cont = 0
    elif numero_juegos == len(df_juegos):
        break
    else:
        continue           
driver.quit()

Funciones

In [3]:
def carga_pagina_inicial():
    from selenium.common.exceptions import TimeoutException
    from fake_useragent import UserAgent
    "fake_user_agent"
    ua = UserAgent()
    timeout = 10
    
    lista_tiempo = [3,3.1,3.2]
    #rechazamos cookies

    try:
        butt_coo = EC.presence_of_element_located((By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img'))
        WebDriverWait(driver, timeout).until(butt_coo)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    rechazar_cookies = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div/div[1]/div/div[2]/button/img')
    rechazar_cookies.click()

    #vamos a la pestaña de explora

    try:
        expl_butt = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a'))
        WebDriverWait(driver, timeout).until(expl_butt)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    explora = driver.find_element(By.XPATH, '/html/body/div[3]/section/div/div/div/ul/li[5]/a')
    explora.click()


    #plat

    try:
        sort_butt = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button'))
        WebDriverWait(driver, timeout).until(sort_butt)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")


    sort_plat = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[4]/button')
    sort_plat.click()


    #ordenamos

    try:
        sort_g = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button'))
        WebDriverWait(driver, timeout).until(sort_g)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")


    sort_gam = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/header/button')
    sort_gam.click()

    #AZ


    try:
        sort_az_ = EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]'))
        WebDriverWait(driver, timeout).until(sort_az_)
    except TimeoutException:
        print("Timed out waiting for sort button to appear")

    sort_az = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section/div/div/div/div[3]/div/div/div/div[3]/div[1]/div/div/span/label[3]')
    sort_az.click()

    # Obtenemos número de juegos para scrapear

    time.sleep(random.choice(lista_tiempo))

    driver.implicitly_wait(10)
    url = driver.current_url
    headers = {'User-Agent': ua.random}
    response = requests.get(url, headers=headers)
    soup = bs(response.text,'html')
    return soup
            
def pagina_concreta_carga(pagina):
    pag = 1
    while pag != pagina:
        next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
        next_page.click()
        pag += 1

In [15]:
### PRUEBA FUNCIONAL WEBSCRAPING DE X JUEGOS Y TIEMPO UTILIZADO ### Completamente funcional, pocos campos, meter más cosas genero etc... (Prueba con timeouts en vez de random choice de tiempos)
# Reset df
from selenium.common.exceptions import TimeoutException
from fake_useragent import UserAgent
"fake_user_agent"
ua = UserAgent()
service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
timeout = 10

df_juegos = pd.DataFrame(columns=["Titulo","Día y hora","Plataforma","Genero","Compañia",
                                "Lanzamiento","Idiomas","Calificación PSN","Número de calificaciones","Calificación 5 estrellas",
                                "Calificación 4 estrellas","Calificación 3 estrellas","Calificación 2 estrellas",
                                "Calificación 1 estrella","Precio original sin PSN","Precio actual sin PSN","Precio original con PSN","Precio actual con PSN"]) #,"Precio con mayor rebaja"

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://store.playstation.com/")


lista_tiempo = [3,3.1,3.2]

carga_pagina_inicial()
# Mostrar un numero de juegos limitado
numero_juegos = 1000
# numero_juegos = soup.find('div', class_ ='psw-t-body psw-c-t-2').get_text() #Si son todos los de la web

#seleccion de juego
page = 1
cont = 0 # Establecemos el primer juego que estará en cont = 1, pero lo establecemos en 0 para iniciarlo
while cont <= 24:
    try:
        cont += 1
        try:
            sel_game = EC.presence_of_element_located((By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a'))
            WebDriverWait(driver, timeout).until(sel_game)
        except TimeoutException:
            print(f"Timed out waiting for game to appear, game number{cont}")
        
        select_game = driver.find_element(By.XPATH, f'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/ul/li[{cont}]/div/a')
        select_game.click()

        
        # Variable por juego
        
        # Titulo
        
        # Info que nos descargamos de la pagina del juego.

        driver.implicitly_wait(10)
        try:  
            url = driver.current_url
            headers = {'User-Agent': ua.random}
            response = requests.get(url, headers=headers)
            soup = bs(response.text,'html')
        except Exception as e:
            print(f"Error al obtener la URL: error en el juego{cont}, página{page}")
            

        try: #Intentamos primero con un tipo de letra
            titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-8 psw-l-line-break-word").get_text()
        except: #Intentamos después con otro tipo de letra
            titulo = soup.find("h1", class_="psw-m-b-5 psw-t-title-l psw-t-size-7 psw-l-line-break-word").get_text()
        
        # Día y hora de webscrappeo
        
        fecha_webs = datetime.now()
        fecha_webs = datetime.isoformat(fecha_webs)
            
        # Precio original sin PSN
        try:
            precio_original_sn_psn = soup.find("span",class_="psw-t-title-s psw-c-t-2 psw-t-strike").get_text() 
        except:
            precio_original_sn_psn = soup.find("span",class_="psw-t-title-m").get_text()
            
        # Precio original con PSN
        try:
            precio_original_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#originalPrice','class':'psw-t-title-s psw-c-t-2 psw-t-strike'}).get_text()
        except: 
            try:
                precio_original_cn_psn = soup.find("span",attrs={'class':'psw-truncate-text-1 psw-p-t-1 psw-l-exclude@desktop'}).get_text()
            except:
                precio_original_cn_psn = precio_original_sn_psn
        # Precio actual sin PSN
        try:
            precio_actual_sn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
        except:
            precio_actual_sn_psn = precio_original_sn_psn
        
        #Precio Actual con PSN
        try:
            precio_actual_cn_psn = soup.find("span",attrs={'data-qa':'mfeCtaMain#offer1#finalPrice','class':'psw-t-title-m psw-m-r-4'}).get_text()
        except: 
            try:
                precio_actual_cn_psn = soup.find("span",class_="psw-t-title-m psw-m-r-4").get_text()
            except: 
                precio_actual_cn_psn = precio_actual_sn_psn
        
        # Plataforma
        try:
            plataforma = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#platform-value'}).get_text()
        except:
            plataforma = "No hay información"
        
        # Genero
        try:
            genero = soup.find("span", attrs={'style':'text-transform: capitalize;'}).get_text()
        except:
            genero = "No hay información"
        
        # Compañia
        try:
            compania = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#publisher-value'}).get_text()
        except:
            compania = "No hay información"
        
        # Lanzamiento
        try:  
            lanzamiento = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#releaseDate-value'}).get_text()
        except:
            lanzamiento = "No hay información"
        
        # Idiomas
        try:
            idiomas = soup.find("dd", attrs={'class':'psw-p-r-6 psw-p-r-0@tablet-s psw-t-bold psw-l-w-1/2 psw-l-w-1/6@tablet-s psw-l-w-1/6@tablet-l psw-l-w-1/8@laptop psw-l-w-1/6@desktop psw-l-w-1/6@max','data-qa':'gameInfo#releaseInformation#subtitles-value'}).get_text()
        except:
            idiomas = "No hay información"
        # Nº Calificaciones
        try:
            num_calificaciones = soup.find("span", attrs={'class':'psw-c-t-2 psw-t-secondary','data-qa':'mfe-star-rating#overall-rating#total-ratings'}).get_text()
        except:
            num_calificaciones = "No hay información"
        # Calificación PSN
        
        try:
            calificacion = soup.find("div", attrs={'class':'psw-t-subtitle psw-t-bold psw-l-line-center','data-qa':'mfe-game-title#average-rating'}).get_text()
        except:
            calificacion = "No hay información"
        try:
            calificacion_1 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress1#percentage-label'}).get_text()
        except:
            calificacion_1 = "No hay información"
        try:
            calificacion_2 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress2#percentage-label'}).get_text()
        except:
            calificacion_2 = "No hay información"
        try:
            calificacion_3 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress3#percentage-label'}).get_text()
        except:
            calificacion_3 = "No hay información"
        try:
            calificacion_4 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress4#percentage-label'}).get_text()
        except:
            calificacion_4 = "No hay información"
        try:
            calificacion_5 = soup.find("span", attrs={'class':'psw-t-body','data-qa':'mfe-star-rating#overall-rating#rating-progress5#percentage-label'}).get_text()
        except:
            calificacion_5 = "No hay información"
        
        # precio_con_mayor_rebaja = "No hay información"
        
        df_juegos.loc[len(df_juegos)] = {"Titulo":titulo,"Día y hora":fecha_webs,"Plataforma":plataforma,"Genero":genero,"Compañia":compania,"Lanzamiento":lanzamiento,
                                        "Idiomas":idiomas,"Calificación PSN":calificacion,"Número de calificaciones":num_calificaciones,
                                        "Calificación 5 estrellas":calificacion_5,
                                        "Calificación 4 estrellas":calificacion_4,"Calificación 3 estrellas":calificacion_3,
                                        "Calificación 2 estrellas":calificacion_2,"Calificación 1 estrella":calificacion_1,
                                        "Precio original sin PSN":precio_original_sn_psn,"Precio actual sin PSN":precio_actual_sn_psn,"Precio original con PSN":precio_original_cn_psn, "Precio actual con PSN":precio_actual_cn_psn}
                                        # "Precio con mayor rebaja":precio_con_mayor_rebaja 
        
        time.sleep(random.choice(lista_tiempo))
        try:  
            driver.back();
        except:
            print(f"Timed out waiting for game to appear, game number{cont}, reinicio captura")
            driver.quit()
            driver = webdriver.Chrome(service=service, options=options)
            service = Service(executable_path='../../psn_env/Lib/site-packages/selenium/webdriver/chrome/chromedriver.exe')
            options = webdriver.ChromeOptions()
            driver.get("https://store.playstation.com/")
            carga_pagina_inicial()
            pagina_concreta_carga(page)
            continue

        
        #Comprobar error aquí
        if cont == 24:
            next_page = driver.find_element(By.XPATH,'/html/body/div[3]/main/div/section/div/div/div/div[2]/div[2]/div/nav/button[2]')  
            next_page.click()
            page += 1      
            cont = 0
            time.sleep(random.choice(lista_tiempo))
        elif numero_juegos == len(df_juegos):
            break
        else:
            continue
    except:
        # Volvemos a hacer la carga completa de la pagina
        driver.quit()
        driver.get("https://store.playstation.com/")
        carga_pagina_inicial()
        pagina_concreta_carga(page)
                  
driver.quit()

MaxRetryError: HTTPConnectionPool(host='localhost', port=60682): Max retries exceeded with url: /session/0d1cf81676154e613ccb4953ddb141a6/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000234AB4F45D0>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

In [13]:
soup

<!DOCTYPE html>
<html class="psw-root psw-rem-base" dir="ltr" lang="es-es"><head><meta content="2.100.0" name="wca-ssr-version"/><meta content="es-ES" name="wca-locale"/><meta content="dark" name="wca-theme"/><script id="wca-config-overrides" type="application/json">
    {"client":{"webCheckout":{"clientId":"2eb25762-877f-4140-b341-7c7e14c19f98"}}}</script>
<link crossorigin="" href="https://static.playstation.com" rel="preconnect"/>
<link href="https://static.playstation.com" rel="dns-prefetch"/>
<link href="https://web.np.playstation.com/api/graphql/v1" rel="preconnect"/>
<link href="https://web.np.playstation.com/api/graphql/v1" rel="dns-prefetch"/>
<link crossorigin="" href="https://image.api.playstation.com" rel="preconnect"/>
<link href="https://image.api.playstation.com" rel="dns-prefetch"/>
<link href="https://id.sonyentertainmentnetwork.com" rel="dns-prefetch"/>
<link href="https://checkout.playstation.com" rel="dns-prefetch"/>
<link href="https://static.playstation.com/font-s

In [9]:
df_juegos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 412 entries, 0 to 411
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Titulo                    412 non-null    object
 1   Día y hora                412 non-null    object
 2   Plataforma                412 non-null    object
 3   Genero                    412 non-null    object
 4   Compañia                  412 non-null    object
 5   Lanzamiento               412 non-null    object
 6   Idiomas                   412 non-null    object
 7   Calificación PSN          412 non-null    object
 8   Número de calificaciones  412 non-null    object
 9   Calificación 5 estrellas  412 non-null    object
 10  Calificación 4 estrellas  412 non-null    object
 11  Calificación 3 estrellas  412 non-null    object
 12  Calificación 2 estrellas  412 non-null    object
 13  Calificación 1 estrella   412 non-null    object
 14  Precio original sin PSN   412 n

[Volver al índice](#0)

In [10]:
df_juegos

,Titulo,Día y hora,Plataforma,Genero,Compañia,Lanzamiento,Idiomas,Calificación PSN,Número de calificaciones,Calificación 5 estrellas,Calificación 4 estrellas,Calificación 3 estrellas,Calificación 2 estrellas,Calificación 1 estrella,Precio original sin PSN,Precio actual sin PSN,Precio original con PSN,Precio actual con PSN
0,0 Degrees,2024-02-05T19:19:59.617722,PS5,No hay información,eastasiasoft,30/12/2021,Inglés,3.87,85 calificaciones,51 %,11 %,24 %,6 %,9 %,"4,99 €","1,49 €",,"1,49 €"
1,1001 Spikes,2024-02-05T19:20:04.003864,PS4,No hay información,Nicalis Inc.,6/10/2015,Inglés,4.43,893 calificaciones,68 %,20 %,6 %,3 %,4 %,"14,99 €","14,99 €","14,99 €","14,99 €"
2,100ft Robot Golf,2024-02-05T19:20:08.806164,PS4,No hay información,NO GOBLIN LLC,18/9/2017,Inglés,3.42,286 calificaciones,38 %,17 %,13 %,12 %,20 %,"9,99 €","9,99 €","9,99 €","9,99 €"
3,101 Ways to Die,2024-02-05T19:20:13.323884,PS4,No hay información,VISION GAMES PUBLISHING,22/3/2016,"Alemán, Español, Francés (Francia), Inglés, It...",3.91,256 calificaciones,46 %,25 %,14 %,4 %,11 %,"9,99 €","9,99 €","9,99 €","9,99 €"
4,10 Second Ninja X,2024-02-05T19:20:17.812512,PS4,No hay información,Curve Digital,19/7/2016,"Alemán, Español, Francés (Francia), Inglés, Ruso",3.52,3.615 calificaciones,47 %,11 %,13 %,6 %,23 %,"9,99 €","9,99 €","9,99 €","9,99 €"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,Angry Birds VR: Isle of Pigs,2024-02-05T20:00:03.931889,PS5,No hay información,RESOLUTION GAMES,10/10/2023,"Alemán, Coreano, Español, Francés (Francia), I...",4.48,131 calificaciones,62 %,27 %,8 %,3 %,0 %,"19,99 €","19,99 €","19,99 €","19,99 €"
408,Angry Video Game Nerd I & II Deluxe,2024-02-05T20:00:13.737338,PS4,No hay información,SCREENWAVE MEDIA,25/2/2021,"Alemán, Español, Francés (Francia), Inglés, It...",4.84,343 calificaciones,90 %,7 %,2 %,1 %,1 %,"9,99 €","9,99 €","9,99 €","9,99 €"
409,Anima: Gate of memories,2024-02-05T20:00:20.346589,PS4,No hay información,ANIMA PROJECT CARLOS BLAS GARCÍA APARICIO,2/6/2016,"Chino (Tradicional), Coreano, Español, Francés...",4.01,920 calificaciones,53 %,21 %,9 %,7 %,10 %,"19,99 €","19,99 €","19,99 €","19,99 €"
410,Anima: Gate of Memories - The Nameless Chronicles,2024-02-05T20:00:27.408482,PS4,No hay información,ANIMA PROJECT CARLOS BLAS GARCÍA APARICIO,18/6/2018,"Español, Francés (Francia), Inglés",4.36,243 calificaciones,70 %,16 %,4 %,2 %,8 %,"19,99 €","19,99 €","19,99 €","19,99 €"


arreglar precios.

Creando base de datos en server elephant

In [1]:
import psycopg2

conn = psycopg2.connect(database="wgjqejjm",
                        host="manny.db.elephantsql.com",
                        user="wgjqejjm",
                        password="EeAurbeM3ZiVV3mq1zeKRkp8_eL_DZUB",
                        port="5432")

In [2]:
cursor = conn.cursor()

In [3]:
table_creation = '''
    CREATE TABLE info_juegos (
        game_id SERIAL PRIMARY KEY,
        game_name TEXT NOT NULL
    )
'''

In [4]:
cursor.execute(table_creation)

In [5]:
conn.commit()
cursor.close()
conn.close()